## Task 1: Import the `ibm_db` Python library

In [ ]:
!pip install ibm_db

     |████████████████████████████████| 798kB 6.1MB/s 
  Created wheel for ibm-db: filename=ibm_db-3.0.3-cp37-cp37m-linux_x86_64.whl size=40998585 sha256=47f9efb95f2bf1bb92894b07bff25804603ace5e531355b5fdc33b0875ea7188
  Stored in directory: /root/.cache/pip/wheels/f2/0e/c0/55fd70ce0709cb4eed1b548ce0367531f931aea30ee0abfaf0
Successfully built ibm-db


In [ ]:
import ibm_db

# Task 2: Identify the database connection credentials

In [ ]:
dsn_hostname = "dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net" # e.g.: "dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net"
dsn_uid = "hwq67488"        # e.g. "abc12345"
dsn_pwd = "88c-68fwtjfkdl5c"      # e.g. "7dBZ3wWt9XN6$o0J"

dsn_driver = "{IBM DB2 ODBC DRIVER}"
dsn_database = "BLUDB"            # e.g. "BLUDB"
dsn_port = "50000"                # e.g. "50000" 
dsn_protocol = "TCPIP"            # i.e. "TCPIP"


## Task 3: Create the database connection


In [ ]:
#DO NOT MODIFY THIS CELL. Just RUN it with Shift + Enter
#Create the dsn connection string
dsn = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={6};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd)

#print the connection string to check correct values are specified
print(dsn)

DRIVER={IBM DB2 ODBC DRIVER};DATABASE=BLUDB;HOSTNAME=dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net;PORT=50000;PROTOCOL=TCPIP;UID=hwq67488;PWD=88c-68fwtjfkdl5c;


In [ ]:
#DO NOT MODIFY THIS CELL. Just RUN it with Shift + Enter
#Create database connection

try:
    conn = ibm_db.connect(dsn, "", "")
    print ("Connected to database: ", dsn_database, "as user: ", dsn_uid, "on host: ", dsn_hostname)

except:
    print ("Unable to connect: ", ibm_db.conn_errormsg() )


Connected to database:  BLUDB as user:  hwq67488 on host:  dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net


## Task 4: Create a table in the database


In [ ]:
#Lets first drop the table INSTRUCTOR in case it exists from a previous attempt
dropQuery = "drop table INSTRUCTOR"

#Now execute the drop statment
dropStmt = ibm_db.exec_immediate(conn, dropQuery)

Exception: ignored

In [ ]:
#Construct the Create Table DDL statement - replace the ... with rest of the statement
createQuery = "create table INSTRUCTOR(ID INTEGER PRIMARY KEY NOT NULL, FNAME VARCHAR(20), LNAME VARCHAR(20), CITY VARCHAR(20), CCODE CHAR(2))"

#Now fill in the name of the method and execute the statement
createStmt = ibm_db.exec_immediate(conn,createQuery)

## Task 5: Insert data into the table


In [ ]:
# We will start by inserting just the first row of data, i.e. for instructor Rav Ahuja

insertQuery = "insert into INSTRUCTOR values (1, 'Rav', 'Ahuja', 'TORONTO', 'CA')"

insertStmt = ibm_db.exec_immediate(conn, insertQuery)

In [ ]:
# Now use a single query to insert the remaining two rows of data

insertQuery2 = "insert into INSTRUCTOR values (2, 'Raul', 'Chong', 'Markham', 'CA'), (3, 'Hima', 'Vasudevan', 'Chicago', 'US')"

insertStmt2 = ibm_db.exec_immediate(conn, insertQuery2)

## Task 6: Query data in the table

In [ ]:
#Construct the query that retrieves all rows from the INSTRUCTOR table
selectQuery = "select * from INSTRUCTOR"

#Execute the statement
selectStmt = ibm_db.exec_immediate(conn, selectQuery)

#Fetch the Dictionary (for the first row only)
ibm_db.fetch_both(selectStmt)

{0: 1,
 1: 'Rav',
 2: 'Ahuja',
 3: 'TORONTO',
 4: 'CA',
 'CCODE': 'CA',
 'CITY': 'TORONTO',
 'FNAME': 'Rav',
 'ID': 1,
 'LNAME': 'Ahuja'}

In [ ]:
#Fetch the rest of the rows and print the ID and FNAME for those rows
while ibm_db.fetch_row(selectStmt) != False:
    print (" ID:",  ibm_db.result(selectStmt, 0), " FNAME:",  ibm_db.result(selectStmt, "FNAME"))

 ID: 2  FNAME: Raul
 ID: 3  FNAME: Hima


In [ ]:
# Bonus: now write and execute an update statement that changes the Rav's CITY to MOOSETOWN

updateQuery = "update INSTRUCTOR set CITY='MOOSETOWN' where FNAME='Rav'"
updateStmt = ibm_db.exec_immediate(conn, updateQuery)

## Task 7: Retrieve data into Pandas

In [ ]:
import pandas
import ibm_db_dbi

In [ ]:
pconn = ibm_db_dbi.Connection(conn)

In [ ]:
#query statement to retrieve all rows in INSTRUCTOR table
selectQuery = "select * from INSTRUCTOR"

#retrieve the query results into a pandas dataframe
pdf = pandas.read_sql(selectQuery, pconn)

#print just the LNAME for first row in the pandas data frame
pdf.LNAME[0]

'Ahuja'

In [ ]:
#print the entire data frame
pdf

,ID,FNAME,LNAME,CITY,CCODE
0,1,Rav,Ahuja,MOOSETOWN,CA
1,2,Raul,Chong,Markham,CA
2,3,Hima,Vasudevan,Chicago,US


## Task 8: Close the Connection

In [ ]:
ibm_db.close(conn)

True